# Session 3 (a): Toy Datasets

First, we perform clustering on a simple dataset to illustrate how clustering works.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn import datasets

n_samples = 500
random_state = 170 # fix it for reproducibility!

# create dataset
dataset = datasets.make_blobs(n_samples=n_samples, cluster_std=[1.0, 1.0, 1.0], random_state=random_state) # Q: What is cluster_std attribute? (always check the documentation!)
X = dataset[0] # data
y = dataset[1] # labels

# visualise dataset
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.show()

# # quick check !
# print(X.shape)
# print(y.shape)

## K-Means

K-Means clusters the points based on the Euclidean distance between them $d_{i,j} = ||\mathbf{x}_i -  \mathbf{x}_j||^2_2$. The goal is to minimise the distance between points in the same cluster, and maximise the distance between points from different clusters.

The objective function is:

\begin{equation}
    \min \left( \sum_{i=0}^n ||\mathbf{x}_i - \sum_{k=0}^{n_k} \delta_{i, k} \boldsymbol{\mu}_k||^2_2\right) \ \ \ k = 1, \dots, n_k
\end{equation}

which is also called the inertia, and where $\delta_{i, k}$ is the cluster assignment associated to the point $x_i$ (0 or 1) and $\mu_k$ is the centroid of the k-th cluster.

The algorithm works iteratively:

0) Initialise the $\mu_k$ centroids
1) Define the cluster assignment $\delta_{i, k}$ for each $x_i$ ($x_i$ is associated with the closest centroid)
2) Minimise the objection function
3) Update $\mu_k$ and go back to (1)


In [ ]:
# To do:
# - center the dataset
# - apply KMeans https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans
# - plot the data with the centroids and the labels found by KMeans

# from ... import ...

# 1) Center the dataset:
# X0 = ...

# 2) Apply kmeans
# kmeans = ...

# centroids = ...
# labels = ...

# 3) Plot
# plt.scatter(...) # plot the data
# plt.scatter(...) # plot the centroids
# plt.legend()
# plt.show()

# Clustering to do Local PCA (LPCA)

The goal of the exercise is to do clustering on a dataset representing a noisy 2D non-linear function.
* How do we select the correct number of clusters?
* Why LPCA works better than PCA on a non-linear dataset?

First thing, we create our synthetic dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits

def non_linear_func_orig(x):
    y = np.sin(x) + np.sin(1.5*x)
    return y

def non_linear_func(x):
    y = non_linear_func_orig(x) + 0.25*np.random.randn(x.size)
    return y

# This code is used to create our synthetic dataset
size = 1000
seed = 42
np.random.seed(seed) # Fix it for reproducibility

x = np.random.rand(size) * 2.6*np.pi - np.pi/2
y = non_linear_func(x)

limits = [np.pi/2.5, 1.2*np.pi]
eps = np.pi/10

mask = ((x < limits[0] + eps) & (x > limits[0] - eps)) | (x < limits[1] + eps) & (x > limits[1] - eps)
x = x[~mask]
y = y[~mask]

# Create the noisy samples dataset
X = np.zeros((x.size, 2))
X[:,0] = x
X[:,1] = y

x_orig = np.linspace(x.min(), x.max(), 200)
y_orig = non_linear_func_orig(x_orig)

# We can plot the dataset and the original non-linear function
plt.scatter(X[:,0], X[:,1], c='b', alpha=0.5) # Noisy samples
plt.plot(x_orig, y_orig, c='cyan', lw=5) # Original function
plt.xlabel('x')
plt.ylabel('y')
plt.show()

# quick check !
# print(X.shape)

## Global PCA
Now we apply the PCA and try to reconstruct the dataset using only one dimension.
 - To do:
    - obtain X0 (dataset centred and scaled with mean and std)
    - apply PCA
    - recontruct the solution with one dimension (q=1)
        - hint:
            - X     = Z  @ A.T
            - X_rec = Zq @ Aq.T
    - plot the reconstruction vs the original dataset


 - hint: the documentation and examples to use the PCA object are at:
       https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
# To do:
# - Apply PCA
# - Reconstruct with a single PC

# from ... import ...

# 1) Apply PCA
# scaler = ... # (or use X_mean and X_std)
# X0 = ...

# pca = ... # don't forget to fit the pca object from sklearn !

# A = ... # loadings
# Z = ... # scores

# 2) Reconstruct with 1 PC
# q = 1
# X0_rec = ...
# X_rec = ...

# Plot the reconstruction VS original
# plt.scatter(X[:,0], X[:,1], c='b', alpha=0.5) # Noisy samples
# plt.plot(x_orig, y_orig, c='cyan', lw=5) # Original
# plt.scatter(X_rec[:,0], X_rec[:,1], c='red', alpha=0.5) # Reconstructed samples
# plt.xlabel('x')
# plt.ylabel('y')
# plt.show()

# Q: Discuss the results! Why does it look bad?

## Clustering

We can try and improve the reconstruction accuracy using a local approac ('piecewise'). The first thing to do is then to cluster the dataset using kmeans.
- To do:
    - Use KMeans object to obtain the cluster labels or indexes (=`idx`) with *init='random'* and *n_init='auto'*
    - Use a number of clusters (k) of your choice.
    - Plot the dataset and colour it by the value of the label

- Hint: the documentation and examples to use the PCA object are at:
       https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
# To do:
# - Apply KMeans to the dataset

# from ... import ...

# 1) Choose the number of clusters
# k = ...

# 2) Apply kmeans to the centred dataset (see above)
# kmeans = ...

# centroids = ...
# labels = ...

# 3) Plot
# plt.scatter(...) # plot the data
# plt.scatter(...) # plot the centroids
# plt.legend()
# plt.show()

## Selection of the clusters' number

In this case we can visually identify the number of clusters, because we have only two dimensions. In higher dimensions, it is not possible to visualise the dataset. Also, we would prefer to have an automatic method to select the correct number of clusters.

We can use the Davies-Boulding score to estimate the number of clusters (https://scikit-learn.org/stable/modules/clustering.html#davies-bouldin-index)

- To do:
    - Calculate the DB score for a range of k (number of clusters) that goes from 2 to 10.
    - Plot the DB score as a function of k.
    - Select n_clusters based on the DB score
   
   
- Hint:
    - the documentation and examples to use the PCA object are at:
    https://scikit-learn.org/stable/modules/generated/sklearn.metrics.davies_bouldin_score.html
    - you can use a 'for' loop to calculate the DB for different k


In [ ]:
# To do:
# Compute and plot the DB index for n_cluster = 2 to 10

# from ... import ...

# 1) Create the range of k and the corresponding DB score
# k_array = ...
# db_score_array = ...

# 2) Plot DB score vs n_clusters
# plt.plot(k_array, db_score_array)
# plt.xlabel('k')
# plt.ylabel('db_score')
# plt.show()

# 3) Select n_clusters based on db_scores
# index = ...
# n_clusters = ...
# print(f'n_clusters = {n_clusters}')

## Local PCA

Now we can now apply the PCA inside each cluster.
- To do:
    - apply PCA for each cluster
    - plot the reconstruction using only the first PC.
- Hint:
    - Use a 'for' loop to apply the PCA in each cluster.
    - to select only the data of a single cluster use a mask:
           To select the data with label 0:
             mask = (idx == 0)
             X_k = X[mask]
    - To get the correct reconstruction, you need to **center and scale** the data **in each cluster** before applying the PCA, and then de-center and de-scale the data

In [ ]:
# To do:
# - Apply PCA to each cluster

# 1) Compute KMeans with the correct number of clusters
# kmeans = ...

# centroids = ...
# labels = ...

# 2) Apply PCA for each cluster

# q = 1

# Create the PCA object
# pca = PCA()

# Loop on clusters
# for k in range(n_clusters):
#     print(f'k = {k}')

#     # Select the points belonging to cluster k using the labels
#     mask = ... # Create the mask
#     X_k = X[mask]

#     # Scale the subdataset
#     ...
#     X0_k = ...

#     # Perform the PCA on this scaled subdataset
#     ...
#     A_k = ... # loadings
#     Z_k = ... # scores

#     # Reconstruct, uncenter and unscale
#     X0_k_rec = ...
#     X_k_rec : ...

#     # Plot the reconstruction
#     plt.scatter(...) # Noisy samples
#     plt.scatter(...) # Reconstructed samples

# Plot the original
# plt.plot(x_orig, y_orig, c='cyan', lw=5, alpha=0.7)
# plt.show()

# Q: Discuss the results! Why does it still look bad?

## Local PCA with VQPCA

We can compare the KMEANS results with VQPCA. To perform VQPCA, we need the [OpenMORe](https://github.com/burn-research/OpenMORe) library. You can run the next cell to install it.
VQPCA is a clustering algorithm that assign points to the clusters based on the reconstruction error, so it tries to find clusters that can be well represented by a linear manifold.

In [ ]:
! git clone https://github.com/burn-research/OpenMORe.git

import sys

sys.path.insert(0,'/content/OpenMORe')

import OpenMORe.clustering as clustering
# import OpenMORe.clustering as clustering

In [ ]:
# This is a dictionary used to set the VQPCA parameters.
settings_clustering = {
    #centering and scaling options
    "center"                    : True,
    "centering_method"          : "mean",
    "scale"                     : True,
    "scaling_method"            : "auto",

    #set the initialization method (random, observations, kmeans, pkcia, uniform)
    "initialization_method"     : "uniform",

    #set the number of clusters and PCs in each cluster
    "number_of_clusters"        : 3,
    "number_of_eigenvectors"    : 1,

    #enable additional options:
    "correction_factor"         : "off",    # --> enable eventual corrective coefficients for the LPCA algorithm:
                                            #     'off', 'c_range', 'uncorrelation', 'local_variance', 'phc_multi', 'local_skewness' are available

    "classify"                  : False,    # --> call the method to classify a new matrix Y on the basis of the lpca clustering
    "write_on_txt"              : False,    # --> write the idx vector containing the label for each observation
    "evaluate_clustering"       : True,     # --> enable the calculation of indeces to evaluate the goodness of the clustering
    "neighbors_number"          : 0,

    "kNN_post"                  : False,     # activate the kNN algorithm once the convergence is achieved
}

# To do: follow the example in
# https://github.com/burn-research/OpenMORe/blob/master/examples/clustering/lpcaExample.py
# to find the vqpca labels, then plot the clusters

# First we create the model and then we fit it to the dataset
model = clustering.lpca(X, settings_clustering)
idx_vqpca = model.fit()

In [ ]:
# Now we can plot the clusters
plt.scatter(X[:,0], X[:,1], c=idx_vqpca)
plt.show()

In [ ]:
# To do:
# - Apply PCA to each cluster

# q = 1

# Create the PCA object
# pca = PCA()

# Loop on clusters
# for k in range(n_clusters):
#     print(f'k = {k}')

#     # Select the points belonging to cluster k using the labels
#     mask = ... # Create the mask
#     X_k = X[mask]

#     # Scale the subdataset
#     ...
#     X0_k = ...

#     # Perform the PCA on this scaled subdataset
#     ...
#     A_k = ... # loadings
#     Z_k = ... # scores

#     # Reconstruct, uncenter and unscale
#     X0_k_rec = ...
#     X_k_rec : ...

#     # Plot the reconstruction
#     plt.scatter(...) # Noisy samples
#     plt.scatter(...) # Reconstructed samples

# Plot the original
# plt.plot(x_orig, y_orig, c='cyan', lw=5, alpha=0.7)
# plt.show()

# Q: Discuss the results! Why does it still look bad?